# Phân đoạn (Segmentation) bệnh trên da xoài

Notebook này trình bày quy trình hoàn chỉnh cho mô hình phân đoạn bệnh trên da xoài, bao gồm các giai đoạn:
1. Chuẩn bị môi trường và cài đặt thư viện
2. Tải và chuẩn bị dữ liệu
3. Khám phá và trực quan hóa dữ liệu
4. Tạo và huấn luyện các mô hình phân đoạn
5. Đánh giá và so sánh mô hình
6. Sử dụng mô hình để dự đoán

Mô hình phân đoạn cho phép phát hiện nhiều loại bệnh cùng lúc trên một quả xoài cũng như xác định chính xác vị trí và diện tích bị nhiễm bệnh.

## 1. Chuẩn bị môi trường và cài đặt thư viện

In [ ]:
# Cài đặt các thư viện cần thiết
!pip install segmentation-models
!pip install albumentations
!pip install opencv-python
!pip install scikit-learn
!pip install matplotlib
!pip install pyyaml
!pip install tensorflow>=2.4.0
!pip install tqdm

In [ ]:
# Import các thư viện cần thiết
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import json
import random
import tensorflow as tf
import yaml
import glob
import segmentation_models as sm
import albumentations as A
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from pathlib import Path
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from datetime import datetime

# Đặt seed cho tính khả tái
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Kiểm tra GPU
print("Tensorflow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))

## 2. Tải và chuẩn bị dữ liệu
### 2.1. Thiết lập cấu trúc thư mục

In [ ]:
# Tạo cấu trúc thư mục cho dữ liệu phân đoạn
def setup_directory_structure(base_dir='data'):
    """Thiết lập cấu trúc thư mục cho dữ liệu phân đoạn."""
    # Tạo thư mục chính
    os.makedirs(base_dir, exist_ok=True)
    
    # Tạo các thư mục con
    for directory in ['raw', 'segmentation/images', 'segmentation/masks', 'segmentation/annotations',
                      'segmentation/train/images', 'segmentation/train/masks',
                      'segmentation/val/images', 'segmentation/val/masks',
                      'segmentation/test/images', 'segmentation/test/masks',
                      'models']:
        os.makedirs(os.path.join(base_dir, directory), exist_ok=True)
    
    print(f"Cấu trúc thư mục đã được tạo tại {base_dir}")

In [ ]:
# Thiết lập cấu trúc thư mục
setup_directory_structure()

### 2.2. Xử lý dữ liệu annotation

In [ ]:
# Thiết lập mapping cho các nhãn bệnh
LABEL_MAPPING = {
    "background": 0,  # Nền (không bệnh)
    "DC": 1,          # Da cám
    "DE": 2,          # Da ếch
    "DD": 3,          # Đóm đen
    "TT": 4,          # Thán thư
    "RD": 5,          # Rùi đụt
}

CLASS_NAMES = ["background", "da_cam", "da_ech", "dom_den", "than_thu", "rui_dut"]

# Màu cho các lớp (RGB)
COLORS = [
    [0, 0, 0],      # Background - đen
    [255, 0, 0],    # Da cám - đỏ
    [0, 255, 0],    # Da ếch - xanh lá
    [0, 0, 255],    # Đóm đen - xanh dương
    [255, 255, 0],  # Thán thư - vàng
    [255, 0, 255]   # Rùi đụt - tím
]

In [ ]:
def process_json_to_mask(json_path, output_size=(512, 512), label_mapping=LABEL_MAPPING):
    """Chuyển đổi file JSON annotation thành mask."""
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # Lấy kích thước ảnh gốc
        img_height = data.get('imageHeight', output_size[0])
        img_width = data.get('imageWidth', output_size[1])
        
        # Tạo mask trống
        mask = np.zeros((img_height, img_width), dtype=np.uint8)
        
        # Vẽ các đa giác lên mask
        for shape in data.get('shapes', []):
            label = shape.get('label')
            points = shape.get('points')
            
            # Chuyển đổi label thành ID nếu có mapping
            label_id = label_mapping.get(label, 1) if label_mapping else 1
            
            # Chuyển đổi points thành định dạng phù hợp cho cv2.fillPoly
            points_array = np.array(points, dtype=np.int32)
            
            # Vẽ polygon
            cv2.fillPoly(mask, [points_array], label_id)
        
        # Resize mask về kích thước mong muốn
        if output_size and (img_height != output_size[0] or img_width != output_size[1]):
            mask = cv2.resize(mask, output_size, interpolation=cv2.INTER_NEAREST)
        
        return mask
    
    except Exception as e:
        print(f"Lỗi khi xử lý file {json_path}: {e}")
        return None

### 2.3. Chuẩn bị dữ liệu từ nguồn

Trong phần này, bạn cần chỉ định nguồn dữ liệu thực tế. 
- Thay `input_dir` bằng đường dẫn thực tế đến dữ liệu của bạn
- Dữ liệu gồm ảnh và file annotation JSON (định dạng Labelme)

In [ ]:
# Chỉ định đường dẫn đến dữ liệu nguồn
input_dir = "path/to/raw/data"  # Thay bằng đường dẫn thực tế
output_dir = "data"
img_size = (512, 512)
val_split = 0.15
test_split = 0.15

In [ ]:
def collect_data_files(input_dir):
    """Thu thập tất cả các cặp file ảnh và json từ thư mục đầu vào."""
    print(f"Đang quét thư mục {input_dir} để tìm file ảnh và annotation...")
    
    image_files = []
    json_files = []
    
    # Duyệt qua tất cả các thư mục và tìm file
    for root, _, files in os.walk(input_dir):
        for file in files:
            file_path = os.path.join(root, file)
            # Thu thập file ảnh
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_files.append(file_path)
            # Thu thập file json
            elif file.lower().endswith('.json'):
                json_files.append(file_path)
    
    print(f"Đã tìm thấy {len(image_files)} file ảnh và {len(json_files)} file annotation.")
    return image_files, json_files

In [ ]:
def match_image_annotation(image_files, json_files):
    """Ghép cặp file ảnh và file annotation."""
    print("Đang ghép cặp file ảnh và annotation...")
    
    # Tạo dict lưu tên file ảnh và đường dẫn
    image_dict = {}
    for img_path in image_files:
        img_name = os.path.basename(img_path)
        image_dict[img_name] = img_path
    
    # Tìm file json tương ứng với mỗi ảnh
    matched_pairs = []
    for json_path in json_files:
        with open(json_path, 'r', encoding='utf-8') as f:
            try:
                json_data = json.load(f)
                # Lấy tên file ảnh từ imagePath trong json
                image_path = json_data.get('imagePath', '')
                if not image_path:
                    continue
                
                # Chuẩn hóa đường dẫn và lấy tên file
                image_name = os.path.basename(image_path.replace('\\', '/'))
                
                # Tìm file ảnh tương ứng
                if image_name in image_dict:
                    matched_pairs.append((image_dict[image_name], json_path))
                else:
                    # Trường hợp tên file trong json không khớp chính xác
                    # Tìm file có tên gần giống
                    potential_matches = [img for img in image_dict.keys() 
                                        if os.path.splitext(img)[0] in os.path.splitext(image_name)[0] 
                                        or os.path.splitext(image_name)[0] in os.path.splitext(img)[0]]
                    if potential_matches:
                        matched_pairs.append((image_dict[potential_matches[0]], json_path))
            except json.JSONDecodeError:
                print(f"Lỗi khi đọc file {json_path}. Bỏ qua.")
    
    print(f"Đã ghép được {len(matched_pairs)} cặp ảnh và annotation.")
    return matched_pairs

In [ ]:
def process_single_pair(img_path, json_path, images_dir, masks_dir, img_size, label_mapping):
    """Xử lý một cặp file ảnh và annotation."""
    try:
        # Lấy tên file gốc
        img_name = os.path.basename(img_path)
        base_name = os.path.splitext(img_name)[0]
        
        # Đọc và resize ảnh
        img = cv2.imread(img_path)
        if img is None:
            print(f"Không thể đọc ảnh {img_path}")
            return
        
        img_resized = cv2.resize(img, img_size, interpolation=cv2.INTER_AREA)
        
        # Tạo mask từ file json
        mask = process_json_to_mask(json_path, img_size, label_mapping)
        if mask is None:
            print(f"Không thể tạo mask từ {json_path}")
            return
        
        # Lưu ảnh và mask
        cv2.imwrite(os.path.join(images_dir, f"{base_name}.jpg"), img_resized)
        cv2.imwrite(os.path.join(masks_dir, f"{base_name}.png"), mask)
        
    except Exception as e:
        print(f"Lỗi khi xử lý {img_path}: {e}")

In [ ]:
def prepare_data():
    # Thu thập và ghép cặp file
    image_files, json_files = collect_data_files(input_dir)
    matched_pairs = match_image_annotation(image_files, json_files)
    
    # Chia dữ liệu thành train, val, test
    random.shuffle(matched_pairs)
    n_total = len(matched_pairs)
    n_test = int(n_total * test_split)
    n_val = int(n_total * val_split)
    n_train = n_total - n_test - n_val
    
    train_pairs = matched_pairs[:n_train]
    val_pairs = matched_pairs[n_train:n_train+n_val]
    test_pairs = matched_pairs[n_train+n_val:]
    
    print(f"Chia dữ liệu: {n_train} train, {n_val} validation, {n_test} test")
    
    # Xử lý từng phần
    for subset, pairs in zip(['train', 'val', 'test'], [train_pairs, val_pairs, test_pairs]):
        images_dir = os.path.join(output_dir, f'segmentation/{subset}/images')
        masks_dir = os.path.join(output_dir, f'segmentation/{subset}/masks')
        
        for img_path, json_path in tqdm(pairs, desc=f"Xử lý {subset}"):
            process_single_pair(
                img_path, 
                json_path, 
                images_dir, 
                masks_dir, 
                img_size, 
                LABEL_MAPPING
            )
    
    # Lưu tất cả vào thư mục raw để tham khảo
    raw_dir = os.path.join(output_dir, 'raw')
    for img_path, json_path in tqdm(matched_pairs, desc="Sao chép raw data"):
        # Sao chép file ảnh
        img_name = os.path.basename(img_path)
        shutil.copy2(img_path, os.path.join(raw_dir, img_name))
        
        # Sao chép file json
        json_name = os.path.basename(json_path)
        shutil.copy2(json_path, os.path.join(raw_dir, json_name))
    
    print("Hoàn thành xử lý dữ liệu!")

In [ ]:
# Thực hiện chuẩn bị dữ liệu
import shutil  # Import thư viện shutil cho việc copy file

# Chỉ chạy khi có dữ liệu thực tế. Bỏ comment dòng dưới khi sử dụng
# prepare_data()

## 3. Khám phá và trực quan hóa dữ liệu

In [ ]:
def visualize_sample(img_path, mask_path):
    """Hiển thị một mẫu dữ liệu với ảnh gốc và mask."""
    # Đọc ảnh và mask
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    
    # Tạo mask màu để hiển thị
    colored_mask = np.zeros((*mask.shape, 3), dtype=np.uint8)
    for class_idx, color in enumerate(COLORS):
        colored_mask[mask == class_idx] = color
    
    # Tạo ảnh overlay
    alpha = 0.6
    overlay = cv2.addWeighted(img, 1-alpha, colored_mask, alpha, 0)
    
    # Hiển thị
    plt.figure(figsize=(16, 5))
    
    plt.subplot(1, 3, 1)
    plt.imshow(img)
    plt.title("Ảnh gốc")
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(colored_mask)
    plt.title("Mask")
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    plt.imshow(overlay)
    plt.title("Overlay")
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Tính tỷ lệ diện tích từng loại bệnh
    total_pixels = mask.size
    class_areas = {}
    
    for class_idx, class_name in enumerate(CLASS_NAMES):
        pixel_count = np.sum(mask == class_idx)
        percentage = (pixel_count / total_pixels) * 100
        class_areas[class_name] = percentage
    
    # Hiển thị tỷ lệ diện tích
    print("Tỷ lệ diện tích từng loại bệnh:")
    for class_name, percentage in class_areas.items():
        if percentage > 0:
            print(f"{class_name}: {percentage:.2f}%")

In [ ]:
def explore_dataset(subset='train'):
    """Khám phá dataset và hiển thị một số mẫu."""
    images_dir = os.path.join(output_dir, f'segmentation/{subset}/images')
    masks_dir = os.path.join(output_dir, f'segmentation/{subset}/masks')
    
    # Lấy danh sách file ảnh
    image_files = sorted(glob.glob(os.path.join(images_dir, "*.jpg")))
    
    if not image_files:
        print(f"Không tìm thấy file ảnh trong {images_dir}")
        return
    
    print(f"Tìm thấy {len(image_files)} ảnh trong tập {subset}")
    
    # Hiển thị một số mẫu ngẫu nhiên
    num_samples = min(5, len(image_files))
    sample_indices = np.random.choice(len(image_files), num_samples, replace=False)
    
    for idx in sample_indices:
        img_path = image_files[idx]
        img_name = os.path.basename(img_path)
        mask_path = os.path.join(masks_dir, os.path.splitext(img_name)[0] + ".png")
        
        print(f"\nMẫu: {img_name}")
        visualize_sample(img_path, mask_path)

In [ ]:
# Khám phá tập huấn luyện
explore_dataset('train')

In [ ]:
# Khám phá tập validation
explore_dataset('val')

# Phân tích phân phối lớp trong dataset
def analyze_class_distribution():
    """Phân tích phân phối các lớp trong dataset."""
    subsets = ['train', 'val', 'test']
    class_distribution = {subset: {class_name: 0 for class_name in CLASS_NAMES} for subset in subsets}
    total_pixels = {subset: 0 for subset in subsets}
    
    for subset in subsets:
        masks_dir = os.path.join(output_dir, f'segmentation/{subset}/masks')
        mask_files = glob.glob(os.path.join(masks_dir, "*.png"))
        
        for mask_path in tqdm(mask_files, desc=f"Phân tích {subset}"):
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            if mask is None:
                continue
                
            total_pixels[subset] += mask.size
            
            for class_idx, class_name in enumerate(CLASS_NAMES):
                pixel_count = np.sum(mask == class_idx)
                class_distribution[subset][class_name] += pixel_count
    
    # Tính tỷ lệ phần trăm và hiển thị kết quả
    plt.figure(figsize=(15, 8))
    
    for i, subset in enumerate(subsets):
        plt.subplot(1, 3, i+1)
        
        percentages = [
            (class_distribution[subset][class_name] / total_pixels[subset] * 100)
            for class_name in CLASS_NAMES if class_distribution[subset][class_name] > 0
        ]
        
        labels = [
            class_name 
            for class_name in CLASS_NAMES 
            if class_distribution[subset][class_name] > 0
        ]
        
        # Sử dụng màu tương ứng cho từng lớp bệnh
        colors = [tuple(c/255 for c in COLORS[i]) for i, class_name in enumerate(labels)]
        
        plt.pie(percentages, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors)
        plt.title(f'Phân phối lớp trong tập {subset}')
    
    plt.tight_layout()
    plt.show()
    
    # Hiển thị bảng số lượng pixel
    print("Số lượng pixel cho từng lớp bệnh:")
    print(f"{'Lớp':<15} {'Train':<15} {'Validation':<15} {'Test':<15}")
    print("-" * 60)
    
    for class_name in CLASS_NAMES:
        train_count = class_distribution['train'][class_name]
        val_count = class_distribution['val'][class_name]
        test_count = class_distribution['test'][class_name]
        
        print(f"{class_name:<15} {train_count:<15} {val_count:<15} {test_count:<15}")

analyze_class_distribution()

In [ ]:
# Hiển thị tất cả các loại bệnh và màu sắc tương ứng
plt.figure(figsize=(15, 3))
for i, (class_name, color) in enumerate(zip(CLASS_NAMES, COLORS)):
    plt.subplot(1, len(CLASS_NAMES), i+1)
    plt.imshow([[color]])
    plt.title(class_name)
    plt.axis('off')
plt.tight_layout()
plt.show()

# Hiển thị các ảnh và mask theo từng loại bệnh
def show_samples_by_disease():
    """Hiển thị các ảnh mẫu theo từng loại bệnh."""
    masks_dir = os.path.join(output_dir, 'segmentation/train/masks')
    images_dir = os.path.join(output_dir, 'segmentation/train/images')
    
    mask_files = glob.glob(os.path.join(masks_dir, "*.png"))
    
    # Lưu các ảnh có từng loại bệnh
    samples_by_disease = {class_name: [] for class_name in CLASS_NAMES[1:]}  # Bỏ qua background
    
    for mask_path in mask_files:
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            continue
        
        base_name = os.path.splitext(os.path.basename(mask_path))[0]
        img_path = os.path.join(images_dir, f"{base_name}.jpg")
        
        for class_idx, class_name in enumerate(CLASS_NAMES[1:], 1):  # Bắt đầu từ lớp thứ 1 (bỏ qua background)
            if np.sum(mask == class_idx) > 1000:  # Có đủ pixel của loại bệnh này
                samples_by_disease[class_name].append((img_path, mask_path))
                break  # Chỉ lấy loại bệnh chính
    
    # Hiển thị mẫu cho từng loại bệnh
    for class_name, samples in samples_by_disease.items():
        if not samples:
            print(f"Không có mẫu cho bệnh {class_name}")
            continue
        
        print(f"\nMẫu cho bệnh {class_name} (tổng số: {len(samples)}):")
        
        # Chọn ngẫu nhiên 1 mẫu
        if len(samples) > 0:
            sample = random.choice(samples)
            visualize_sample(sample[0], sample[1])

# Hiển thị mẫu theo từng loại bệnh
show_samples_by_disease()

In [ ]:
# Tạo mask màu từ mask grayscale
def create_colored_mask(mask):
    """Tạo mask màu từ mask grayscale."""
    colored_mask = np.zeros((*mask.shape, 3), dtype=np.uint8)
    for class_idx, color in enumerate(COLORS):
        colored_mask[mask == class_idx] = color
    return colored_mask

## 4. Tạo và huấn luyện các mô hình phân đoạn

In [ ]:
# Tạo cấu hình mô hình segmentation
segmentation_config = {
    "data": {
        "train_dir": os.path.join(output_dir, "segmentation/train"),
        "validation_dir": os.path.join(output_dir, "segmentation/val"),
        "test_dir": os.path.join(output_dir, "segmentation/test"),
        "img_size": list(img_size),
        "use_augmentation": True
    },
    "model": {
        "input_shape": [*img_size, 3],
        "num_classes": len(CLASS_NAMES),
        "save_dir": os.path.join(output_dir, "models"),
        "class_names": CLASS_NAMES,
        "segmentation_model": {
            "architecture": "unet",  # Hoặc "fpn", "pspnet", "deeplabv3"
            "encoder": "resnet34",   # Hoặc "resnet50", "efficientnetb0", "mobilenetv2"
            "encoder_weights": "imagenet",
            "activation": "softmax"
        }
    },
    "training": {
        "batch_size": 8,
        "epochs": 50,
        "learning_rate": 0.0001,
        "early_stopping_patience": 10,
        "reduce_lr_patience": 5,
        "use_augmentation": True,
        "class_weights": None,
        "loss": "categorical_crossentropy",
        "optimizer": "adam"
    }
}

# Lưu cấu hình
os.makedirs(os.path.join(output_dir, "configs"), exist_ok=True)
with open(os.path.join(output_dir, "configs", "segmentation_config.yaml"), "w") as f:
    yaml.dump(segmentation_config, f, default_flow_style=False)

### 4.1. Tạo bộ nạp dữ liệu

In [ ]:
# Bộ nạp dữ liệu cho segmentation
class SegmentationDataGenerator(tf.keras.utils.Sequence):
    """Bộ nạp dữ liệu cho model segmentation."""
    
    def __init__(self, images_dir, masks_dir, batch_size=8, img_size=(512, 512), 
                num_classes=6, augmentation=False, augmentation_config=None, shuffle=True):
        """Khởi tạo generator."""
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.batch_size = batch_size
        self.img_size = img_size
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.augmentation = augmentation
        
        # Lấy danh sách file ảnh
        self.image_paths = sorted(glob.glob(os.path.join(images_dir, "*.jpg")))
        
        # Lấy danh sách file mask tương ứng
        self.mask_paths = []
        for img_path in self.image_paths:
            img_name = os.path.basename(img_path)
            base_name = os.path.splitext(img_name)[0]
            mask_path = os.path.join(masks_dir, f"{base_name}.png")
            if os.path.exists(mask_path):
                self.mask_paths.append(mask_path)
            else:
                # Nếu không tìm thấy mask tương ứng, loại bỏ ảnh khỏi danh sách
                self.image_paths.remove(img_path)
        
        # Tạo albumentation cho augmentation
        if augmentation:
            self.aug_transform = A.Compose([
                A.HorizontalFlip(p=0.5),
                A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
                A.GaussianBlur(blur_limit=(3, 7), p=0.3),
                A.RandomScale(scale_limit=0.2, p=0.5),
                A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=15, p=0.5),
                A.RandomCrop(height=img_size[0], width=img_size[1], p=0.7)
            ])
        
        # Tạo indices
        self.indices = np.arange(len(self.image_paths))
        self.on_epoch_end()
    
    def __len__(self):
        """Trả về số batch trong một epoch."""
        return len(self.image_paths) // self.batch_size
    
    def __getitem__(self, index):
        """Trả về một batch dữ liệu."""
        # Lấy indices của batch hiện tại
        indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        
        # Khởi tạo batch data
        batch_imgs = np.zeros((self.batch_size, *self.img_size, 3), dtype=np.float32)
        batch_masks = np.zeros((self.batch_size, *self.img_size, self.num_classes), dtype=np.float32)
        
        # Nạp dữ liệu
        for i, idx in enumerate(indices):
            # Đọc ảnh và mask
            img = cv2.imread(self.image_paths[idx])
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, self.img_size)
            
            mask = cv2.imread(self.mask_paths[idx], cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, self.img_size, interpolation=cv2.INTER_NEAREST)
            
            # Áp dụng augmentation nếu được yêu cầu
            if self.augmentation:
                augmented = self.aug_transform(image=img, mask=mask)
                img = augmented['image']
                mask = augmented['mask']
            
            # Chuẩn hóa ảnh
            img = img / 255.0
            
            # One-hot encoding cho mask
            mask_onehot = to_categorical(mask, num_classes=self.num_classes)
            
            # Thêm vào batch
            batch_imgs[i] = img
            batch_masks[i] = mask_onehot
        
        return batch_imgs, batch_masks
    
    def on_epoch_end(self):
        """Được gọi khi kết thúc một epoch."""
        if self.shuffle:
            np.random.shuffle(self.indices)

In [ ]:
def create_data_generators():
    """Tạo generators cho training, validation và test."""
    # Lấy config
    data_config = segmentation_config['data']
    training_config = segmentation_config['training']
    
    # Đường dẫn đến dữ liệu
    train_images_dir = os.path.join(data_config['train_dir'], 'images')
    train_masks_dir = os.path.join(data_config['train_dir'], 'masks')
    val_images_dir = os.path.join(data_config['validation_dir'], 'images')
    val_masks_dir = os.path.join(data_config['validation_dir'], 'masks')
    test_images_dir = os.path.join(data_config['test_dir'], 'images')
    test_masks_dir = os.path.join(data_config['test_dir'], 'masks')
    
    # Số lớp và kích thước ảnh
    num_classes = segmentation_config['model']['num_classes']
    img_size = tuple(data_config['img_size'])
    batch_size = training_config['batch_size']
    use_augmentation = training_config['use_augmentation']
    
    # Tạo generator
    train_gen = SegmentationDataGenerator(
        train_images_dir, 
        train_masks_dir, 
        batch_size=batch_size, 
        img_size=img_size, 
        num_classes=num_classes, 
        augmentation=use_augmentation
    )
    
    val_gen = SegmentationDataGenerator(
        val_images_dir, 
        val_masks_dir, 
        batch_size=batch_size, 
        img_size=img_size, 
        num_classes=num_classes, 
        augmentation=False
    )
    
    test_gen = SegmentationDataGenerator(
        test_images_dir, 
        test_masks_dir, 
        batch_size=batch_size, 
        img_size=img_size, 
        num_classes=num_classes, 
        augmentation=False,
        shuffle=False
    )
    
    return train_gen, val_gen, test_gen

### 4.2. Định nghĩa mô hình

In [ ]:
def build_segmentation_model():
    """Xây dựng mô hình phân đoạn."""
    # Lấy config
    model_config = segmentation_config['model']
    segmentation_model_config = model_config['segmentation_model']
    
    # Thiết lập framework cho segmentation-models
    sm.set_framework('tf.keras')
    
    # Tùy chọn mô hình
    input_shape = tuple(model_config['input_shape'])
    num_classes = model_config['num_classes']
    architecture = segmentation_model_config['architecture']
    encoder = segmentation_model_config['encoder']
    encoder_weights = segmentation_model_config.get('encoder_weights', 'imagenet')
    activation = segmentation_model_config.get('activation', 'softmax')
    
    # Chọn kiến trúc mô hình
    if architecture.lower() == 'unet':
        model_fn = sm.Unet
    elif architecture.lower() == 'fpn':
        model_fn = sm.FPN
    elif architecture.lower() == 'pspnet':
        model_fn = sm.PSPNet
    elif architecture.lower() == 'deeplabv3':
        model_fn = sm.DeepLabV3
    elif architecture.lower() == 'linknet':
        model_fn = sm.Linknet
    else:
        raise ValueError(f"Kiến trúc {architecture} không được hỗ trợ")
    
    # Xây dựng mô hình
    model = model_fn(
        encoder_name=encoder,
        encoder_weights=encoder_weights,
        classes=num_classes,
        activation=activation,
        input_shape=input_shape
    )
    
    return model

# Tạo các metrics
def get_segmentation_metrics():
    """Trả về các metrics phù hợp cho mô hình phân đoạn."""
    return [
        sm.metrics.IOUScore(threshold=0.5),  # IoU score
        sm.metrics.FScore(threshold=0.5),    # F1 score
        'accuracy'
    ]

# Tạo hàm loss
def get_segmentation_loss(loss_name='categorical_crossentropy', class_weights=None):
    """Trả về hàm loss phù hợp cho mô hình phân đoạn."""
    if loss_name == 'categorical_crossentropy':
        return 'categorical_crossentropy'
    elif loss_name == 'dice_loss':
        return sm.losses.DiceLoss(class_weights=class_weights)
    elif loss_name == 'focal_loss':
        return sm.losses.CategoricalFocalLoss()
    elif loss_name == 'jaccard_loss':
        return sm.losses.JaccardLoss(class_weights=class_weights)
    elif loss_name == 'combined_loss':
        dice_loss = sm.losses.DiceLoss(class_weights=class_weights)
        focal_loss = sm.losses.CategoricalFocalLoss()
        return dice_loss + focal_loss
    else:
        raise ValueError(f"Loss {loss_name} không được hỗ trợ")

### 4.3. Huấn luyện mô hình

In [ ]:
def train_segmentation_model():
    """Huấn luyện mô hình phân đoạn."""
    # Tạo generators
    train_gen, val_gen, _ = create_data_generators()
    
    # Tạo mô hình
    model = build_segmentation_model()
    
    # Lấy config
    training_config = segmentation_config['training']
    model_config = segmentation_config['model']
    
    # Tạo optimizer
    if training_config['optimizer'].lower() == 'adam':
        optimizer = Adam(learning_rate=training_config['learning_rate'])
    elif training_config['optimizer'].lower() == 'rmsprop':
        optimizer = RMSprop(learning_rate=training_config['learning_rate'])
    elif training_config['optimizer'].lower() == 'sgd':
        optimizer = SGD(learning_rate=training_config['learning_rate'], momentum=0.9)
    else:
        optimizer = Adam(learning_rate=training_config['learning_rate'])
    
    # Biên dịch mô hình
    model.compile(
        optimizer=optimizer,
        loss=get_segmentation_loss(training_config['loss'], training_config['class_weights']),
        metrics=get_segmentation_metrics()
    )
    
    # In thông tin mô hình
    model.summary()
    
    # Tạo các callbacks
    callbacks = []
    
    # Model checkpoint
    os.makedirs(model_config['save_dir'], exist_ok=True)
    model_path = os.path.join(model_config['save_dir'], 'segmentation_model.h5')
    callbacks.append(ModelCheckpoint(
        model_path,
        monitor='val_iou_score',
        mode='max',
        save_best_only=True,
        verbose=1
    ))
    
    # Early stopping
    callbacks.append(EarlyStopping(
        monitor='val_loss',
        patience=training_config['early_stopping_patience'],
        restore_best_weights=True,
        verbose=1
    ))
    
    # Reduce LR on plateau
    callbacks.append(ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=training_config['reduce_lr_patience'],
        min_lr=1e-7,
        verbose=1
    ))
    
    # TensorBoard
    if training_config.get('use_tensorboard', True):
        log_dir = os.path.join(output_dir, 'logs', 'segmentation', datetime.now().strftime("%Y%m%d-%H%M%S"))
        os.makedirs(log_dir, exist_ok=True)
        callbacks.append(TensorBoard(
            log_dir=log_dir,
            histogram_freq=1,
            update_freq='epoch'
        ))
    
    # Huấn luyện mô hình
    print(f"Bắt đầu huấn luyện mô hình {segmentation_config['model']['segmentation_model']['architecture']} với encoder {segmentation_config['model']['segmentation_model']['encoder']}...")
    
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=training_config['epochs'],
        callbacks=callbacks,
        verbose=1
    )
    
    # Lưu lịch sử huấn luyện
    save_training_plots(history, model_config['save_dir'])
    
    return model, history

In [ ]:
def save_training_plots(history, save_dir):
    """Lưu biểu đồ quá trình huấn luyện."""
    os.makedirs(save_dir, exist_ok=True)
    
    # Tạo biểu đồ loss
    plt.figure(figsize=(16, 6))
    
    # Loss
    plt.subplot(1, 3, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # IoU Score
    plt.subplot(1, 3, 2)
    plt.plot(history.history['iou_score'], label='Train IoU')
    plt.plot(history.history['val_iou_score'], label='Validation IoU')
    plt.title('IoU Score')
    plt.xlabel('Epoch')
    plt.ylabel('IoU Score')
    plt.legend()
    
    # F1 Score
    plt.subplot(1, 3, 3)
    plt.plot(history.history['f1-score'], label='Train F1')
    plt.plot(history.history['val_f1-score'], label='Validation F1')
    plt.title('F1 Score')
    plt.xlabel('Epoch')
    plt.ylabel('F1 Score')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, 'segmentation_training_history.png'))
    plt.close()
    
    # Lưu lịch sử huấn luyện vào file
    history_file = os.path.join(save_dir, 'segmentation_history.json')
    import json
    with open(history_file, 'w') as f:
        json.dump({
            'loss': history.history['loss'],
            'val_loss': history.history['val_loss'],
            'iou_score': history.history['iou_score'],
            'val_iou_score': history.history['val_iou_score'],
            'f1-score': history.history['f1-score'],
            'val_f1-score': history.history['val_f1-score'],
            'accuracy': history.history['accuracy'],
            'val_accuracy': history.history['val_accuracy']
        }, f)
    
    print(f"Đã lưu lịch sử huấn luyện vào {history_file}")

# Chỉ huấn luyện nếu có dữ liệu thực tế
# Bỏ comment dòng dưới khi muốn huấn luyện
# model, history = train_segmentation_model()

### 4.4. Thử nghiệm nhiều kiến trúc mô hình khác nhau

In [ ]:
def experiment_with_models():
    """Thử nghiệm với nhiều kiến trúc và encoder khác nhau."""
    # Tạo generator cho dữ liệu
    train_gen, val_gen, _ = create_data_generators()
    
    # Danh sách các kiến trúc muốn thử nghiệm
    architectures = ['unet', 'fpn', 'pspnet']
    
    # Danh sách các encoder muốn thử nghiệm
    encoders = ['resnet34', 'efficientnetb0', 'mobilenetv2']
    
    # Cấu hình huấn luyện nhanh cho thử nghiệm
    segmentation_config['training']['epochs'] = 15
    segmentation_config['training']['early_stopping_patience'] = 5
    
    # Kết quả thử nghiệm
    results = []
    
    for architecture in architectures:
        for encoder in encoders:
            print(f"\nThử nghiệm: {architecture} với encoder {encoder}")
            
            # Cập nhật cấu hình
            segmentation_config['model']['segmentation_model']['architecture'] = architecture
            segmentation_config['model']['segmentation_model']['encoder'] = encoder
            
            # Tạo mô hình
            model = build_segmentation_model()
            
            # Lấy config
            training_config = segmentation_config['training']
            
            # Tạo optimizer
            optimizer = Adam(learning_rate=training_config['learning_rate'])
            
            # Biên dịch mô hình
            model.compile(
                optimizer=optimizer,
                loss=get_segmentation_loss(training_config['loss'], training_config['class_weights']),
                metrics=get_segmentation_metrics()
            )
            
            # Early stopping để ngừng sớm nếu không cải thiện
            early_stopping = EarlyStopping(
                monitor='val_loss',
                patience=training_config['early_stopping_patience'],
                restore_best_weights=True,
                verbose=1
            )
            
            # Reduce LR
            reduce_lr = ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=training_config['reduce_lr_patience'],
                min_lr=1e-7,
                verbose=1
            )
            
            # Huấn luyện
            start_time = datetime.now()
            history = model.fit(
                train_gen,
                validation_data=val_gen,
                epochs=training_config['epochs'],
                callbacks=[early_stopping, reduce_lr],
                verbose=1
            )
            end_time = datetime.now()
            
            # Lưu kết quả
            best_val_loss = min(history.history['val_loss'])
            best_val_iou = max(history.history['val_iou_score'])
            training_time = (end_time - start_time).total_seconds() / 60.0  # Thời gian theo phút
            
            results.append({
                'architecture': architecture,
                'encoder': encoder,
                'best_val_loss': best_val_loss,
                'best_val_iou': best_val_iou,
                'epochs_trained': len(history.history['loss']),
                'training_time_minutes': training_time
            })
            
            # Lưu mô hình
            model_save_path = os.path.join(
                segmentation_config['model']['save_dir'], 
                f"{architecture}_{encoder}_model.h5"
            )
            model.save(model_save_path)
            print(f"Đã lưu mô hình vào {model_save_path}")
            
            # Giải phóng bộ nhớ
            del model
            tf.keras.backend.clear_session()
    
    # Hiển thị kết quả
    print("\nKết quả thử nghiệm:\n")
    print(f"{'Architecture':<10} {'Encoder':<15} {'Best Val Loss':<15} {'Best Val IoU':<15} {'Epochs':<10} {'Time (min)':<10}")
    print("-" * 80)
    
    for result in results:
        print(f"{result['architecture']:<10} {result['encoder']:<15} {result['best_val_loss']:<15.4f} {result['best_val_iou']:<15.4f} {result['epochs_trained']:<10} {result['training_time_minutes']:<10.2f}")
    
    # Vẽ biểu đồ so sánh
    plt.figure(figsize=(12, 8))
    
    # Bar chart cho IoU Score
    plt.subplot(2, 1, 1)
    x = np.arange(len(results))
    labels = [f"{r['architecture']}_{r['encoder']}" for r in results]
    plt.bar(x, [r['best_val_iou'] for r in results])
    plt.xticks(x, labels, rotation=45)
    plt.ylabel('Best Validation IoU')
    plt.title('So sánh IoU Score giữa các mô hình')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Bar chart cho thời gian huấn luyện
    plt.subplot(2, 1, 2)
    plt.bar(x, [r['training_time_minutes'] for r in results])
    plt.xticks(x, labels, rotation=45)
    plt.ylabel('Thời gian huấn luyện (phút)')
    plt.title('So sánh thời gian huấn luyện')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.savefig(os.path.join(segmentation_config['model']['save_dir'], 'model_comparison.png'))
    plt.show()
    
    return results

# Thử nghiệm với nhiều mô hình (bỏ comment để chạy)
# experiment_results = experiment_with_models()

## 5. Đánh giá mô hình

In [ ]:
def evaluate_segmentation_model(model_path=None):
    """Đánh giá mô hình phân đoạn trên tập test."""
    # Tạo generator cho dữ liệu test
    _, _, test_gen = create_data_generators()
    
    # Tải mô hình nếu đường dẫn được cung cấp
    if model_path:
        print(f"Đang tải mô hình từ {model_path}...")
        model = tf.keras.models.load_model(
            model_path,
            custom_objects={
                'iou_score': sm.metrics.IOUScore(threshold=0.5),
                'f1-score': sm.metrics.FScore(threshold=0.5)
            }
        )
    else:
        # Nếu không có đường dẫn, tạo mô hình mới (chỉ cho ví dụ)
        print("Không có đường dẫn mô hình được cung cấp. Đang tạo mô hình mới...")
        model = build_segmentation_model()
        
        # Biên dịch mô hình
        training_config = segmentation_config['training']
        optimizer = Adam(learning_rate=training_config['learning_rate'])
        model.compile(
            optimizer=optimizer,
            loss=get_segmentation_loss(training_config['loss']),
            metrics=get_segmentation_metrics()
        )
    
    # Đánh giá mô hình
    print("Đang đánh giá mô hình...")
    results = model.evaluate(test_gen, verbose=1)
    
    # In kết quả
    metrics_names = model.metrics_names
    for name, value in zip(metrics_names, results):
        print(f"{name}: {value:.4f}")
    
    # Lưu kết quả
    save_dir = os.path.join(output_dir, "evaluation_results")
    os.makedirs(save_dir, exist_ok=True)
    
    with open(os.path.join(save_dir, "evaluation_results.txt"), "w") as f:
        for name, value in zip(metrics_names, results):
            f.write(f"{name}: {value:.4f}\n")
    
    # Hiển thị một số ví dụ
    visualize_predictions(model, test_gen, save_dir)
    
    return results

In [ ]:
def visualize_predictions(model, test_gen, save_dir):
    """Hiển thị kết quả dự đoán trên một số mẫu."""
    # Lấy một batch dữ liệu từ generator
    batch_imgs, batch_masks = test_gen[0]
    
    # Dự đoán trên batch
    batch_preds = model.predict(batch_imgs)
    
    # Chuyển đổi từ one-hot encoding về class index
    batch_masks_argmax = np.argmax(batch_masks, axis=-1)
    batch_preds_argmax = np.argmax(batch_preds, axis=-1)
    
    # Số mẫu để hiển thị
    num_samples = min(5, len(batch_imgs))
    
    plt.figure(figsize=(15, 4 * num_samples))
    
    for i in range(num_samples):
        # Ảnh gốc
        plt.subplot(num_samples, 3, 3*i+1)
        plt.imshow(batch_imgs[i])
        plt.title("Ảnh gốc")
        plt.axis('off')
        
        # Mask thực tế
        plt.subplot(num_samples, 3, 3*i+2)
        colored_mask = create_colored_mask(batch_masks_argmax[i])
        plt.imshow(colored_mask)
        plt.title("Mask thực tế")
        plt.axis('off')
        
        # Mask dự đoán
        plt.subplot(num_samples, 3, 3*i+3)
        colored_pred = create_colored_mask(batch_preds_argmax[i])
        plt.imshow(colored_pred)
        plt.title("Mask dự đoán")
        plt.axis('off')
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, "prediction_examples.png"))
    plt.show()
    
    # Tính toán ma trận nhầm lẫn cho từng lớp
    num_classes = segmentation_config['model']['num_classes']
    conf_matrices = []
    
    for class_idx in range(num_classes):
        # Tạo mask nhị phân cho lớp class_idx
        true_masks = (batch_masks_argmax.flatten() == class_idx).astype(int)
        pred_masks = (batch_preds_argmax.flatten() == class_idx).astype(int)
        
        # Tính ma trận nhầm lẫn
        from sklearn.metrics import confusion_matrix
        cm = confusion_matrix(true_masks, pred_masks, labels=[0, 1])
        conf_matrices.append(cm)
    
    # Hiển thị ma trận nhầm lẫn cho từng lớp
    plt.figure(figsize=(15, 10))
    
    for i, (cm, class_name) in enumerate(zip(conf_matrices, CLASS_NAMES)):
        plt.subplot(2, 3, i+1)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
        plt.title(f'Confusion Matrix - {class_name}')
        plt.xlabel('Predicted')
        plt.ylabel('True')
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, "confusion_matrices.png"))
    plt.show()
    
    # Tính các metric cho từng lớp
    from sklearn.metrics import precision_score, recall_score, f1_score
    
    class_metrics = []
    
    for class_idx in range(num_classes):
        # Tạo mask nhị phân cho lớp class_idx
        true_masks = (batch_masks_argmax.flatten() == class_idx).astype(int)
        pred_masks = (batch_preds_argmax.flatten() == class_idx).astype(int)
        
        # Tính các metric
        precision = precision_score(true_masks, pred_masks, zero_division=0)
        recall = recall_score(true_masks, pred_masks, zero_division=0)
        f1 = f1_score(true_masks, pred_masks, zero_division=0)
        
        class_metrics.append({
            'class': class_name,
            'precision': precision,
            'recall': recall,
            'f1_score': f1
        })
    
    # Hiển thị bảng metric cho từng lớp
    print("\nMetrics cho từng lớp:\n")
    print(f"{'Class':<15} {'Precision':<15} {'Recall':<15} {'F1 Score':<15}")
    print("-" * 60)
    
    for metrics in class_metrics:
        print(f"{metrics['class']:<15} {metrics['precision']:<15.4f} {metrics['recall']:<15.4f} {metrics['f1_score']:<15.4f}")
    
    # Lưu vào file
    with open(os.path.join(save_dir, "class_metrics.txt"), "w") as f:
        f.write(f"{'Class':<15} {'Precision':<15} {'Recall':<15} {'F1 Score':<15}\n")
        f.write("-" * 60 + "\n")
        
        for metrics in class_metrics:
            f.write(f"{metrics['class']:<15} {metrics['precision']:<15.4f} {metrics['recall']:<15.4f} {metrics['f1_score']:<15.4f}\n")

# Đánh giá mô hình (bỏ comment để chạy)
# evaluate_results = evaluate_segmentation_model('path/to/your/segmentation_model.h5')

## 6. Sử dụng mô hình

In [ ]:
def predict_segmentation(image_path, model_path, output_path=None, overlay=True):
    """
    Dự đoán phân đoạn cho một ảnh.
    
    Args:
        image_path: Đường dẫn đến ảnh cần dự đoán
        model_path: Đường dẫn đến mô hình đã huấn luyện
        output_path: Đường dẫn lưu kết quả (tùy chọn)
        overlay: Có tạo overlay hay không
        
    Returns:
        pred_mask: Mask dự đoán
        overlay_img: Ảnh overlay nếu overlay=True
    """
    # Lấy kích thước ảnh từ config
    img_size = tuple(segmentation_config['model']['input_shape'][:2])
    
    # Đọc ảnh
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Không thể đọc ảnh từ {image_path}")
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Resize ảnh
    img_resized = cv2.resize(img, img_size)
    
    # Chuẩn bị đầu vào
    img_input = img_resized / 255.0
    img_input = np.expand_dims(img_input, axis=0)
    
    # Tải mô hình
    model = tf.keras.models.load_model(
        model_path,
        custom_objects={
            'iou_score': sm.metrics.IOUScore(threshold=0.5),
            'f1-score': sm.metrics.FScore(threshold=0.5)
        }
    )
    
    # Dự đoán
    pred = model.predict(img_input)[0]
    pred_mask = np.argmax(pred, axis=-1)
    
    # Tạo mask màu
    colored_mask = create_colored_mask(pred_mask)
    
    # Tạo overlay
    overlay_img = None
    if overlay:
        alpha = 0.6
        overlay_img = cv2.addWeighted(img_resized, 1-alpha, colored_mask, alpha, 0)
    
    # Lưu kết quả nếu cần
    if output_path:
        output_img = overlay_img if overlay else colored_mask
        plt.imsave(output_path, output_img)
        print(f"Đã lưu kết quả dự đoán vào {output_path}")
    
    # Hiển thị kết quả
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.imshow(img_resized)
    plt.title("Ảnh gốc")
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(colored_mask)
    plt.title("Mask dự đoán")
    plt.axis('off')
    
    if overlay:
        plt.subplot(1, 3, 3)
        plt.imshow(overlay_img)
        plt.title("Overlay")
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Tính phần trăm diện tích cho từng lớp bệnh
    total_pixels = pred_mask.size
    class_areas = {}
    
    for class_idx, class_name in enumerate(CLASS_NAMES):
        pixel_count = np.sum(pred_mask == class_idx)
        percentage = (pixel_count / total_pixels) * 100
        class_areas[class_name] = percentage
    
    # Hiển thị phần trăm diện tích
    print("Phần trăm diện tích của từng loại bệnh:")
    for class_name, percentage in class_areas.items():
        if percentage > 0:
            print(f"{class_name}: {percentage:.2f}%")
    
    return pred_mask, overlay_img if overlay else colored_mask